In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras import backend as k
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import numpy as np
import h5py

In [ ]:
img_width, img_height = 256, 256

### Build the network
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=(256, 256, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block3_pool'))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block4_pool'))

# TODO Use this block ?
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block5_pool'))

model.summary()

In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
a = [layer.name for layer in model.layers]
print(a)

In [ ]:
weights_path = './vgg19_weights.h5' 
# ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5)
f = h5py.File(weights_path)
model.load_weights(weights_path, by_name=True)

layer_count=0;
weights_dict = []
for layer in model.layers:
    weights = layer.get_weights()
    weights_dict.append(weights)
    layer_count = layer_count + 1
    print("Model Layer name : {}".format(str(layer.name)))

print("The total number layers is : " + str(layer_count))
for i in layer_dict.keys():
    index = a.index(i)
    model.layers[index].set_weights(weights_dict[index])

In [ ]:

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
batch_size = 16
adam = Adam(lr=0.001)
model.compile(loss='mse', optimizer=adam, metrics=['accuracy'])

filepath="./models_vgg/checkpoint_model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history_object = model.fit_generator(datagen.flow(X_train, y_train_one_hot, batch_size=batch_size),
                                     validation_data=datagen_validation.flow(X_train, y_train_one_hot, batch_size=32),
                                     samples_per_epoch=len(X_train),
                                     epochs=60,
                                     verbose=1,
                                     validation_steps=int(len(X_train) / batch_size),
                                     callbacks=callbacks_list)

In [ ]:
model_path = 'models_vgg/first_model.h5'
model.save(model_path)
print("Model saved to {}".format(model_path))